# Install necessary libraries

In [1]:
!nvidia-smi
!pip install -q supervision "ultralytics<=8.3.40"
import cv2

import numpy as np
import supervision as sv

from tqdm import tqdm
from ultralytics import YOLO
from supervision.assets import VideoAssets, download_assets
from collections import defaultdict, deque

Sat Jul 12 12:51:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# cars only in polygone zone annotated
import cv2
import numpy as np
from collections import defaultdict, deque
from ultralytics import YOLO
from tqdm import tqdm



# Constants
SOURCE_VIDEO_PATH = "/content/drive/MyDrive/IU of Applied Sciences - Master of Artificial Intelligence/Master Thesis with Mushyam, Aditya, Dr./CODES and archive/Speed Estimation Project/Data/IMG_3575.mov"  # Update your path
TARGET_VIDEO_PATH = "vehicles-result.mp4"
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5
MODEL_NAME = "yolov8s.pt"
MODEL_RESOLUTION = 1280

# Source points for perspective transform
SOURCE = np.array([
    [845, 372],
    [1435, 372],
    [1789, 803],
    [659, 803]
])

# Ground truth in meters
TARGET_WIDTH = 7.5
TARGET_HEIGHT = 20

TARGET = np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
])

class ViewTransformer:
    def __init__(self, source: np.ndarray, target: np.ndarray):
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)
        self.m_inv = cv2.getPerspectiveTransform(target, source)  # For reverse transforms

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points
        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

# Initialize video capture
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
if not cap.isOpened():
    raise ValueError("Could not open video file")

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(TARGET_VIDEO_PATH, fourcc, fps, (width, height))

# Initialize YOLO model
model = YOLO(MODEL_NAME)
view_transformer = ViewTransformer(source=SOURCE, target=TARGET)

# Tracking variables
active_trackers = set()
was_in_zone = {}
counted_vehicles = set()
exit_counter = 0
last_vehicles = []
all_speeds = []
coordinates = defaultdict(lambda: deque(maxlen=fps * 2))  # Store coordinates for 2 seconds

# Process video
pbar = tqdm(total=total_frames, desc="Processing Video")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    pbar.update(1)
    annotated_frame = frame.copy()

    # Vehicle detection with tracking
    results = model.track(
        frame,
        persist=True,
        classes=[2, 5, 7],  # Car, bus, truck
        conf=CONFIDENCE_THRESHOLD,
        iou=IOU_THRESHOLD,
        imgsz=MODEL_RESOLUTION,
        verbose=False
    )

    # Extract detections
    detections = results[0].boxes
    boxes = detections.xyxy.cpu().numpy()
    tracker_ids = detections.id.cpu().numpy() if detections.id is not None else []
    confidences = detections.conf.cpu().numpy()
    class_ids = detections.cls.cpu().numpy()

    # Check which detections are in the polygon zone
    current_in_zone = []
    points = []
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        bottom_center = np.array([[(x1 + x2) / 2, y2]])
        in_zone = cv2.pointPolygonTest(SOURCE, (bottom_center[0, 0], bottom_center[0, 1]), False) >= 0
        current_in_zone.append(in_zone)
        points.append(bottom_center[0])

    points = np.array(points)
    world_points = view_transformer.transform_points(points).astype(int)

    # Update tracking status and count exits
    for i, tracker_id in enumerate(tracker_ids):
        if current_in_zone[i]:
            active_trackers.add(tracker_id)
            was_in_zone[tracker_id] = True
        elif tracker_id in active_trackers:
            if was_in_zone.get(tracker_id, False) and tracker_id not in counted_vehicles:
                exit_counter += 1
                counted_vehicles.add(tracker_id)

                # Calculate average speed when vehicle exits
                if len(coordinates.get(tracker_id, [])) >= fps / 2:
                    coordinate_start = coordinates[tracker_id][-1]
                    coordinate_end = coordinates[tracker_id][0]
                    distance = abs(coordinate_start - coordinate_end)
                    time = len(coordinates[tracker_id]) / fps
                    avg_speed = distance / time * 3.6
                    all_speeds.append(avg_speed)
                    last_vehicles.append((tracker_id, int(avg_speed)))
                    if len(last_vehicles) > 5:
                        last_vehicles.pop(0)

            was_in_zone[tracker_id] = False

    # Store detections position (only while in zone)
    for i, tracker_id in enumerate(tracker_ids):
        if current_in_zone[i]:
            coordinates[tracker_id].append(world_points[i, 1])  # Store y-coordinate

    # Annotate frame
    for i, (box, tracker_id) in enumerate(zip(boxes, tracker_ids)):
        x1, y1, x2, y2 = map(int, box)
        conf = confidences[i]
        cls_id = class_ids[i]

        # Draw bounding box
        color = (0, 255, 0)  # Green
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)

        # Prepare label
        if not current_in_zone[i]:
            label = f"#{tracker_id}"
        elif len(coordinates.get(tracker_id, [])) < fps / 2:
            label = f"#{tracker_id}"
        else:
            coordinate_start = coordinates[tracker_id][-1]
            coordinate_end = coordinates[tracker_id][0]
            distance = abs(coordinate_start - coordinate_end)
            time = len(coordinates[tracker_id]) / fps
            speed = distance / time * 3.6
            label = f"#{tracker_id} {int(speed)} km/h"

        # Draw label
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
        cv2.rectangle(annotated_frame, (x1, y1 - text_height - 10), (x1 + text_width + 10, y1), color, -1)
        cv2.putText(annotated_frame, label, (x1 + 5, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

        # Draw trace
        if tracker_id in coordinates and len(coordinates[tracker_id]) > 1:
            trace_points = []
            for y_coord in coordinates[tracker_id]:
                # Transform back to image coordinates
                world_point = np.array([[points[i, 0], y_coord]], dtype=np.float32)
                img_point = cv2.perspectiveTransform(world_point.reshape(-1, 1, 2), view_transformer.m_inv)
                trace_points.append((int(img_point[0, 0, 0]), int(img_point[0, 0, 1])))

            for j in range(1, len(trace_points)):
                cv2.line(annotated_frame, trace_points[j-1], trace_points[j], (0, 255, 255), 2)

    # Draw polygon zone
    cv2.polylines(annotated_frame, [SOURCE.astype(int)], True, (255, 0, 0), 2)

    # Display counters
    text_scale = 1.2
    text_thickness = 2
    text_color = (255, 255, 255)
    text_bg_color = (0, 0, 255)
    line_height = 40
    margin = 30
    start_y = margin + 20

    # Calculate average speed
    avg_all_speed = int(np.mean(all_speeds)) if all_speeds else 0
    counter_text = f"Vehicles: {exit_counter} | Avg Speed: {avg_all_speed} km/h"

    # Draw counter
    (text_width, text_height), _ = cv2.getTextSize(counter_text, cv2.FONT_HERSHEY_SIMPLEX, text_scale, text_thickness)
    text_x = width - text_width - margin
    cv2.rectangle(annotated_frame, (text_x - 10, start_y - text_height - 10), (text_x + text_width + 10, start_y + 10), text_bg_color, -1)
    cv2.putText(annotated_frame, counter_text, (text_x, start_y), cv2.FONT_HERSHEY_SIMPLEX, text_scale, text_color, text_thickness, cv2.LINE_AA)

    # Draw last vehicles
    start_y += line_height
    for i, (vid, speed) in enumerate(reversed(last_vehicles)):
        vehicle_text = f"#{vid}: {speed} km/h"
        (v_text_width, v_text_height), _ = cv2.getTextSize(vehicle_text, cv2.FONT_HERSHEY_SIMPLEX, text_scale*0.8, text_thickness-1)
        v_text_x = width - v_text_width - margin
        cv2.rectangle(annotated_frame, (v_text_x - 10, start_y - v_text_height - 5), (v_text_x + v_text_width + 10, start_y + 5), (50, 50, 150), -1)
        cv2.putText(annotated_frame, vehicle_text, (v_text_x, start_y), cv2.FONT_HERSHEY_SIMPLEX, text_scale*0.8, text_color, text_thickness-1, cv2.LINE_AA)
        start_y += line_height
        if i >= 4: break

    # Write frame
    out.write(annotated_frame)

pbar.close()
cap.release()
out.release()

Processing Video:  99%|█████████▉| 785/793 [01:41<00:01,  7.75it/s]


In [5]:
# all cars annotated
import cv2
import numpy as np
from collections import defaultdict, deque
from ultralytics import YOLO
from tqdm import tqdm

# Constants
SOURCE_VIDEO_PATH = "/content/drive/MyDrive/IU of Applied Sciences - Master of Artificial Intelligence/Master Thesis with Mushyam, Aditya, Dr./CODES and archive/Speed Estimation Project/Data/IMG_3575.mov"  # Update your path
TARGET_VIDEO_PATH = "vehicles-result.mp4"
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5
MODEL_NAME = "yolov8s.pt"
MODEL_RESOLUTION = 1280

# Source points for perspective transform
SOURCE = np.array([
    [845, 372],
    [1435, 372],
    [1789, 803],
    [659, 803]
])

# Ground truth in meters
TARGET_WIDTH = 7.5
TARGET_HEIGHT = 20

TARGET = np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
])

class ViewTransformer:
    def __init__(self, source: np.ndarray, target: np.ndarray):
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)
        self.m_inv = cv2.getPerspectiveTransform(target, source)  # For reverse transforms

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points
        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

# Initialize video capture
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
if not cap.isOpened():
    raise ValueError("Could not open video file")

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(TARGET_VIDEO_PATH, fourcc, fps, (width, height))

# Initialize YOLO model
model = YOLO(MODEL_NAME)
view_transformer = ViewTransformer(source=SOURCE, target=TARGET)

# Tracking variables
active_trackers = set()
was_in_zone = {}
counted_vehicles = set()
exit_counter = 0
last_vehicles = []
all_speeds = []
coordinates = defaultdict(lambda: deque(maxlen=fps * 2))  # Store coordinates for 2 seconds

# Process video
pbar = tqdm(total=total_frames, desc="Processing Video")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    pbar.update(1)
    annotated_frame = frame.copy()

    # Vehicle detection with tracking
    results = model.track(
        frame,
        persist=True,
        classes=[2, 5, 7],  # Car, bus, truck
        conf=CONFIDENCE_THRESHOLD,
        iou=IOU_THRESHOLD,
        imgsz=MODEL_RESOLUTION,
        verbose=False
    )

    # Extract detections
    detections = results[0].boxes
    boxes = detections.xyxy.cpu().numpy()
    tracker_ids = detections.id.cpu().numpy() if detections.id is not None else []
    confidences = detections.conf.cpu().numpy()
    class_ids = detections.cls.cpu().numpy()

    # Check which detections are in the polygon zone
    current_in_zone = []
    points = []
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        bottom_center = np.array([[(x1 + x2) / 2, y2]])
        in_zone = cv2.pointPolygonTest(SOURCE, (bottom_center[0, 0], bottom_center[0, 1]), False) >= 0
        current_in_zone.append(in_zone)
        points.append(bottom_center[0])

    points = np.array(points)
    world_points = view_transformer.transform_points(points).astype(int)

    # Update tracking status and count exits
    for i, tracker_id in enumerate(tracker_ids):
        if current_in_zone[i]:
            active_trackers.add(tracker_id)
            was_in_zone[tracker_id] = True
        elif tracker_id in active_trackers:
            if was_in_zone.get(tracker_id, False) and tracker_id not in counted_vehicles:
                exit_counter += 1
                counted_vehicles.add(tracker_id)

                # Calculate average speed when vehicle exits
                if len(coordinates.get(tracker_id, [])) >= fps / 2:
                    coordinate_start = coordinates[tracker_id][-1]
                    coordinate_end = coordinates[tracker_id][0]
                    distance = abs(coordinate_start - coordinate_end)
                    time = len(coordinates[tracker_id]) / fps
                    avg_speed = distance / time * 3.6
                    all_speeds.append(avg_speed)
                    last_vehicles.append((tracker_id, int(avg_speed)))
                    if len(last_vehicles) > 5:
                        last_vehicles.pop(0)

            was_in_zone[tracker_id] = False

    # Store detections position (only while in zone)
    for i, tracker_id in enumerate(tracker_ids):
        if current_in_zone[i]:
            coordinates[tracker_id].append(world_points[i, 1])  # Store y-coordinate

    # Annotate frame
    for i, (box, tracker_id) in enumerate(zip(boxes, tracker_ids)):
        x1, y1, x2, y2 = map(int, box)
        conf = confidences[i]
        cls_id = class_ids[i]

        # Draw bounding box
        color = (0, 255, 0)  # Green
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)

        # Prepare label
        if not current_in_zone[i]:
            label = f"#{tracker_id}"
        elif len(coordinates.get(tracker_id, [])) < fps / 2:
            label = f"#{tracker_id}"
        else:
            coordinate_start = coordinates[tracker_id][-1]
            coordinate_end = coordinates[tracker_id][0]
            distance = abs(coordinate_start - coordinate_end)
            time = len(coordinates[tracker_id]) / fps
            speed = distance / time * 3.6
            label = f"#{tracker_id} {int(speed)} km/h"

        # Draw label
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
        cv2.rectangle(annotated_frame, (x1, y1 - text_height - 10), (x1 + text_width + 10, y1), color, -1)
        cv2.putText(annotated_frame, label, (x1 + 5, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

        # Draw trace
        if tracker_id in coordinates and len(coordinates[tracker_id]) > 1:
            trace_points = []
            for y_coord in coordinates[tracker_id]:
                # Transform back to image coordinates
                world_point = np.array([[points[i, 0], y_coord]], dtype=np.float32)
                img_point = cv2.perspectiveTransform(world_point.reshape(-1, 1, 2), view_transformer.m_inv)
                trace_points.append((int(img_point[0, 0, 0]), int(img_point[0, 0, 1])))

            for j in range(1, len(trace_points)):
                cv2.line(annotated_frame, trace_points[j-1], trace_points[j], (0, 255, 255), 2)

    # Draw polygon zone with dimensions
    cv2.polylines(annotated_frame, [SOURCE.astype(int)], True, (255, 0, 0), 2)

    # Add width and height annotations near the polygon
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.7
    font_thickness = 2
    text_color = (255, 255, 255)
    bg_color = (0, 0, 255)

    # Calculate positions for the text
# Calculate positions for the text
    top_center = (int((SOURCE[0][0] + SOURCE[1][0]) / 2), int((SOURCE[0][1] + SOURCE[1][1]) / 2))
    left_center = (int((SOURCE[0][0] + SOURCE[3][0]) / 2), int((SOURCE[0][1] + SOURCE[3][1]) / 2))
    right_center = (int((SOURCE[1][0] + SOURCE[2][0]) / 2), int((SOURCE[1][1] + SOURCE[2][1]) / 2))
    bottom_center = (int((SOURCE[2][0] + SOURCE[3][0]) / 2), int((SOURCE[2][1] + SOURCE[3][1]) / 2))

    # Draw width annotation (top side)
    width_text = f"{TARGET_WIDTH}m"
    (wt_width, wt_height), _ = cv2.getTextSize(width_text, font, font_scale, font_thickness)
    cv2.rectangle(annotated_frame,
                 (top_center[0] - wt_width//2 - 5, top_center[1] - wt_height - 5),
                 (top_center[0] + wt_width//2 + 5, top_center[1] + 5),
                 bg_color, -1)
    cv2.putText(annotated_frame, width_text,
               (top_center[0] - wt_width//2, top_center[1]),
               font, font_scale, text_color, font_thickness)

    # Draw height annotation (left side)
    height_text = f"{TARGET_HEIGHT}m"
    (ht_width, ht_height), _ = cv2.getTextSize(height_text, font, font_scale, font_thickness)
    cv2.rectangle(annotated_frame,
                 (left_center[0] - ht_width - 10, left_center[1] - ht_height//2),
                 (left_center[0] - 5, left_center[1] + ht_height//2 + 5),
                 bg_color, -1)
    cv2.putText(annotated_frame, height_text,
               (left_center[0] - ht_width - 5, left_center[1] + ht_height//2),
               font, font_scale, text_color, font_thickness)

    # Display counters
    text_scale = 1.2
    text_thickness = 2
    text_color = (255, 255, 255)
    text_bg_color = (0, 0, 255)
    line_height = 40
    margin = 30
    start_y = margin + 20

    # Calculate average speed
    avg_all_speed = int(np.mean(all_speeds)) if all_speeds else 0
    counter_text = f"Vehicles: {exit_counter} | Avg Speed: {avg_all_speed} km/h"

    # Draw counter
    (text_width, text_height), _ = cv2.getTextSize(counter_text, cv2.FONT_HERSHEY_SIMPLEX, text_scale, text_thickness)
    text_x = width - text_width - margin
    cv2.rectangle(annotated_frame, (text_x - 10, start_y - text_height - 10), (text_x + text_width + 10, start_y + 10), text_bg_color, -1)
    cv2.putText(annotated_frame, counter_text, (text_x, start_y), cv2.FONT_HERSHEY_SIMPLEX, text_scale, text_color, text_thickness, cv2.LINE_AA)

    # Draw last vehicles
    start_y += line_height
    for i, (vid, speed) in enumerate(reversed(last_vehicles)):
        vehicle_text = f"#{vid}: {speed} km/h"
        (v_text_width, v_text_height), _ = cv2.getTextSize(vehicle_text, cv2.FONT_HERSHEY_SIMPLEX, text_scale*0.8, text_thickness-1)
        v_text_x = width - v_text_width - margin
        cv2.rectangle(annotated_frame, (v_text_x - 10, start_y - v_text_height - 5), (v_text_x + v_text_width + 10, start_y + 5), (50, 50, 150), -1)
        cv2.putText(annotated_frame, vehicle_text, (v_text_x, start_y), cv2.FONT_HERSHEY_SIMPLEX, text_scale*0.8, text_color, text_thickness-1, cv2.LINE_AA)
        start_y += line_height
        if i >= 4: break

    # Write frame
    out.write(annotated_frame)

pbar.close()
cap.release()
out.release()

Processing Video:  99%|█████████▉| 785/793 [01:37<00:00,  8.02it/s]


In [7]:
#first 30 seconds of the video
import cv2
import numpy as np
from collections import defaultdict, deque
from ultralytics import YOLO
from tqdm import tqdm

# Constants
SOURCE_VIDEO_PATH = "/content/drive/MyDrive/Arxiv and others/Project/New/IMG_3575.MOV"  # Update your path

TARGET_VIDEO_PATH = "vehicles-result.mp4"
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5
MODEL_NAME = "yolov8s.pt"
MODEL_RESOLUTION = 1280

# Source points for perspective transform
SOURCE = np.array([
    [845, 372],
    [1435, 372],
    [1789, 803],
    [659, 803]
])

# Ground truth in meters
TARGET_WIDTH = 7.5
TARGET_HEIGHT = 20

TARGET = np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
])

class ViewTransformer:
    def __init__(self, source: np.ndarray, target: np.ndarray):
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)
        self.m_inv = cv2.getPerspectiveTransform(target, source)  # For reverse transforms

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points
        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

# Initialize video capture
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
if not cap.isOpened():
    raise ValueError("Could not open video file")

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))

frames_to_process = fps * 30  # 30 seconds worth of frames

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(TARGET_VIDEO_PATH, fourcc, fps, (width, height))

# Initialize YOLO model
model = YOLO(MODEL_NAME)
view_transformer = ViewTransformer(source=SOURCE, target=TARGET)

# Tracking variables
active_trackers = set()
was_in_zone = {}
counted_vehicles = set()
exit_counter = 0
last_vehicles = []
all_speeds = []
coordinates = defaultdict(lambda: deque(maxlen=fps * 2))  # Store coordinates for 2 seconds

# Process video
pbar = tqdm(total=total_frames, desc="Processing Video")

frame_count = 0
while cap.isOpened() and frame_count < frames_to_process:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    pbar.update(1)
    annotated_frame = frame.copy()

    # Vehicle detection with tracking
    results = model.track(
        frame,
        persist=True,
        classes=[2, 5, 7],  # Car, bus, truck
        conf=CONFIDENCE_THRESHOLD,
        iou=IOU_THRESHOLD,
        imgsz=MODEL_RESOLUTION,
        verbose=False
    )

    # Extract detections
    detections = results[0].boxes
    boxes = detections.xyxy.cpu().numpy()
    tracker_ids = detections.id.cpu().numpy() if detections.id is not None else []
    confidences = detections.conf.cpu().numpy()
    class_ids = detections.cls.cpu().numpy()

    # Check which detections are in the polygon zone
    current_in_zone = []
    points = []
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        bottom_center = np.array([[(x1 + x2) / 2, y2]])
        in_zone = cv2.pointPolygonTest(SOURCE, (bottom_center[0, 0], bottom_center[0, 1]), False) >= 0
        current_in_zone.append(in_zone)
        points.append(bottom_center[0])

    points = np.array(points)
    world_points = view_transformer.transform_points(points).astype(int)

    # Update tracking status and count exits
    for i, tracker_id in enumerate(tracker_ids):
        if current_in_zone[i]:
            active_trackers.add(tracker_id)
            was_in_zone[tracker_id] = True
        elif tracker_id in active_trackers:
            if was_in_zone.get(tracker_id, False) and tracker_id not in counted_vehicles:
                exit_counter += 1
                counted_vehicles.add(tracker_id)

                # Calculate average speed when vehicle exits
                if len(coordinates.get(tracker_id, [])) >= fps / 2:
                    coordinate_start = coordinates[tracker_id][-1]
                    coordinate_end = coordinates[tracker_id][0]
                    distance = abs(coordinate_start - coordinate_end)
                    time = len(coordinates[tracker_id]) / fps
                    avg_speed = distance / time * 3.6
                    all_speeds.append(avg_speed)
                    last_vehicles.append((tracker_id, int(avg_speed)))
                    if len(last_vehicles) > 5:
                        last_vehicles.pop(0)

            was_in_zone[tracker_id] = False

    # Store detections position (only while in zone)
    for i, tracker_id in enumerate(tracker_ids):
        if current_in_zone[i]:
            coordinates[tracker_id].append(world_points[i, 1])  # Store y-coordinate

    # Annotate frame
    for i, (box, tracker_id) in enumerate(zip(boxes, tracker_ids)):
        x1, y1, x2, y2 = map(int, box)
        conf = confidences[i]
        cls_id = class_ids[i]

        # Draw bounding box
        color = (0, 255, 0)  # Green
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)

        # Prepare label
        if not current_in_zone[i]:
            label = f"#{tracker_id}"
        elif len(coordinates.get(tracker_id, [])) < fps / 2:
            label = f"#{tracker_id}"
        else:
            coordinate_start = coordinates[tracker_id][-1]
            coordinate_end = coordinates[tracker_id][0]
            distance = abs(coordinate_start - coordinate_end)
            time = len(coordinates[tracker_id]) / fps
            speed = distance / time * 3.6
            label = f"#{tracker_id} {int(speed)} km/h"

        # Draw label
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
        cv2.rectangle(annotated_frame, (x1, y1 - text_height - 10), (x1 + text_width + 10, y1), color, -1)
        cv2.putText(annotated_frame, label, (x1 + 5, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

        # Draw trace
        if tracker_id in coordinates and len(coordinates[tracker_id]) > 1:
            trace_points = []
            for y_coord in coordinates[tracker_id]:
                # Transform back to image coordinates
                world_point = np.array([[points[i, 0], y_coord]], dtype=np.float32)
                img_point = cv2.perspectiveTransform(world_point.reshape(-1, 1, 2), view_transformer.m_inv)
                trace_points.append((int(img_point[0, 0, 0]), int(img_point[0, 0, 1])))

            for j in range(1, len(trace_points)):
                cv2.line(annotated_frame, trace_points[j-1], trace_points[j], (0, 255, 255), 2)

    # Draw polygon zone with dimensions
    cv2.polylines(annotated_frame, [SOURCE.astype(int)], True, (255, 0, 0), 2)

    # Add width and height annotations near the polygon
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.7
    font_thickness = 2
    text_color = (255, 255, 255)
    bg_color = (0, 0, 255)

    # Calculate positions for the text
# Calculate positions for the text
    top_center = (int((SOURCE[0][0] + SOURCE[1][0]) / 2), int((SOURCE[0][1] + SOURCE[1][1]) / 2))
    left_center = (int((SOURCE[0][0] + SOURCE[3][0]) / 2), int((SOURCE[0][1] + SOURCE[3][1]) / 2))
    right_center = (int((SOURCE[1][0] + SOURCE[2][0]) / 2), int((SOURCE[1][1] + SOURCE[2][1]) / 2))
    bottom_center = (int((SOURCE[2][0] + SOURCE[3][0]) / 2), int((SOURCE[2][1] + SOURCE[3][1]) / 2))

    # Draw width annotation (top side)
    width_text = f"{TARGET_WIDTH}m"
    (wt_width, wt_height), _ = cv2.getTextSize(width_text, font, font_scale, font_thickness)
    cv2.rectangle(annotated_frame,
                 (top_center[0] - wt_width//2 - 5, top_center[1] - wt_height - 5),
                 (top_center[0] + wt_width//2 + 5, top_center[1] + 5),
                 bg_color, -1)
    cv2.putText(annotated_frame, width_text,
               (top_center[0] - wt_width//2, top_center[1]),
               font, font_scale, text_color, font_thickness)

    # Draw height annotation (left side)
    height_text = f"{TARGET_HEIGHT}m"
    (ht_width, ht_height), _ = cv2.getTextSize(height_text, font, font_scale, font_thickness)
    cv2.rectangle(annotated_frame,
                 (left_center[0] - ht_width - 10, left_center[1] - ht_height//2),
                 (left_center[0] - 5, left_center[1] + ht_height//2 + 5),
                 bg_color, -1)
    cv2.putText(annotated_frame, height_text,
               (left_center[0] - ht_width - 5, left_center[1] + ht_height//2),
               font, font_scale, text_color, font_thickness)

    # Display counters
    text_scale = 1.2
    text_thickness = 2
    text_color = (255, 255, 255)
    text_bg_color = (0, 0, 255)
    line_height = 40
    margin = 30
    start_y = margin + 20

    # Calculate average speed
    avg_all_speed = int(np.mean(all_speeds)) if all_speeds else 0
    counter_text = f"Vehicles: {exit_counter} | Avg Speed: {avg_all_speed} km/h"

    # Draw counter
    (text_width, text_height), _ = cv2.getTextSize(counter_text, cv2.FONT_HERSHEY_SIMPLEX, text_scale, text_thickness)
    text_x = width - text_width - margin
    cv2.rectangle(annotated_frame, (text_x - 10, start_y - text_height - 10), (text_x + text_width + 10, start_y + 10), text_bg_color, -1)
    cv2.putText(annotated_frame, counter_text, (text_x, start_y), cv2.FONT_HERSHEY_SIMPLEX, text_scale, text_color, text_thickness, cv2.LINE_AA)

    # Draw last vehicles
    start_y += line_height
    for i, (vid, speed) in enumerate(reversed(last_vehicles)):
        vehicle_text = f"#{vid}: {speed} km/h"
        (v_text_width, v_text_height), _ = cv2.getTextSize(vehicle_text, cv2.FONT_HERSHEY_SIMPLEX, text_scale*0.8, text_thickness-1)
        v_text_x = width - v_text_width - margin
        cv2.rectangle(annotated_frame, (v_text_x - 10, start_y - v_text_height - 5), (v_text_x + v_text_width + 10, start_y + 5), (50, 50, 150), -1)
        cv2.putText(annotated_frame, vehicle_text, (v_text_x, start_y), cv2.FONT_HERSHEY_SIMPLEX, text_scale*0.8, text_color, text_thickness-1, cv2.LINE_AA)
        start_y += line_height
        if i >= 4: break

    # Write frame
    out.write(annotated_frame)

pbar.close()
cap.release()
out.release()

Processing Video:  11%|█▏        | 870/7700 [01:47<14:02,  8.11it/s]
